In [1]:
import pandas as pd
import tweepy
import numpy as np
from textblob import TextBlob

import pickle

from datetime import datetime
from datetime import timedelta
from dataclasses import make_dataclass

import re
import emoji
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

import pandas_datareader as web
import matplotlib.pyplot as plt
import mplfinance as mpf
import seaborn as sns
import bitfinex as btf
import time

import nltk
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

compT, compR, compG = [], [],[]
negT, negR, negG = [], [],[]
posT, posR, posG = [], [],[]
neuT, neuR, neuG = [], [],[]
SIAT,SIAR, SIAG = 0, 0, 0

from sklearn.naive_bayes import GaussianNB

import matplotlib.pyplot as plt
import mplfinance as mpf
import seaborn as sns
import bitfinex as btf
import time

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk import ngrams
from collections import Counter

import spacy 
from spacy import displacy

from GoogleNews import GoogleNews
from newspaper import Article
from newspaper import Config

import praw


In [2]:
#Crear una funcion para guardar puntajes de sentimiento con vaderSentiment
def getSIA(text):
  sia = SentimentIntensityAnalyzer()
  sentiment = sia.polarity_scores(text)
  return sentiment
def setSIA(field,pos,neg,neu,comp):
    for i in range(0, len(field)):
      SIA = getSIA(field[i])
      comp.append(SIA['compound'])
      neg.append(SIA['neg'])
      neu.append(SIA['neu'])
      pos.append(SIA['pos'])


In [3]:
def labeleador(df):
    labels=[]
    for i in range(len(df)):
        if df['Open'][i]:
            if df['Open'][i]!="" and df['Close'][i]!="":
                if int(float(df['Open'][i]))<int(float(df['Close'][i])):
                    labels.append(1)
                else:
                    labels.append(0)
    df['Label']=labels
    return df

In [4]:
def limpiaVacios(df):
    for i in range(len(df)):
        if "nameUser" in df['nameUser'][i]:
            df = df.drop(index=i)
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    return df
def limpiaErroresR(df):
    for i in range(len(df)):
        if "Title" in df['Title'][i]:
            df = df.drop(index=i)
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    return df
def limpiaErroresG(df):
    for i in range(len(df)):
        if "Title" in df['Title'][i]:
            df = df.drop(index=i)
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    return df

In [5]:
df_tweet = pd.read_csv('exportTweetData.csv')
df_market = pd.read_csv('exportMarketData.csv')
df_reddit = pd.read_csv('exportRedditData.csv')
df_google = pd.read_csv('exportNewsData.csv')


In [6]:
dataSetCombinadoTwitter = df_tweet.merge(df_market, how='inner', on='Date')
dataSetCombinadoReddit = df_reddit.merge(df_market,how='inner', on='Date')
dataSetCombinadoGoogle = df_google.merge(df_market, how='inner',on='Date')

In [7]:
dataSetCombinadoTwitter = labeleador(dataSetCombinadoTwitter)

dataSetCombinadoReddit = limpiaVacios(dataSetCombinadoReddit)
dataSetCombinadoReddit = labeleador(dataSetCombinadoReddit)
dataSetCombinadoReddit = limpiaErroresR(dataSetCombinadoReddit)

dataSetCombinadoGoogle = limpiaErroresG(dataSetCombinadoGoogle)

dataSetCombinadoGoogle = labeleador(dataSetCombinadoGoogle)


In [8]:
setSIA(dataSetCombinadoTwitter['Tweets'],posT,negT,neuT,compT)
setSIA(dataSetCombinadoReddit['Title'],posR,negR,neuR,compR)
setSIA(dataSetCombinadoGoogle['Desc'],posG,negG,neuG,compG)

dataSetCombinadoTwitter['Compound'] = compT
dataSetCombinadoTwitter['Negative'] = negT
dataSetCombinadoTwitter['Neutral'] = neuT
dataSetCombinadoTwitter['Positive'] = posT

dataSetCombinadoReddit['Compound'] = compR
dataSetCombinadoReddit['Negative'] = negR
dataSetCombinadoReddit['Neutral'] = neuR
dataSetCombinadoReddit['Positive'] = posR

dataSetCombinadoGoogle['Compound'] = compG
dataSetCombinadoGoogle['Negative'] = negG
dataSetCombinadoGoogle['Neutral'] = neuG
dataSetCombinadoGoogle['Positive'] = posG

In [90]:
keep_columns = ['Open','High','Low','Volume','Compound','Negative','Neutral','Positive','Label']
trainDFLinealTwitter = dataSetCombinadoTwitter[keep_columns]
trainDFLinealReddit = dataSetCombinadoReddit[keep_columns]
trainDFLinealGoogle = dataSetCombinadoGoogle[keep_columns]

In [91]:
#Para entrenamiento de programa con modelo lineal
#Para modelo Lineal necesitamos np.array
X_twitter = trainDFLinealTwitter
X_twitter = np.array(X_twitter.drop('Label',1))

#Create the target data set
Y_twitter = np.array(dataSetCombinadoTwitter['Label'])

In [92]:
#Para entrenamiento de programa con modelo lineal
#Para modelo Lineal necesitamos np.array
X_reddit = trainDFLinealReddit
X_reddit = np.array(X_reddit.drop('Label',1))

#Create the target data set
Y_reddit = np.array(dataSetCombinadoReddit['Label'])

In [93]:
#Para entrenamiento de programa con modelo lineal
#Para modelo Lineal necesitamos np.array
X_google = trainDFLinealGoogle
X_google = np.array(X_google.drop('Label',1))

#Create the target data set
Y_google = np.array(dataSetCombinadoGoogle['Label'])

In [94]:
#Set de entrenamiento Twitter
x_train_t, x_test_t, y_train_t, y_test_t = train_test_split(X_twitter,Y_twitter, test_size = 0.2, random_state = 0)

In [95]:
modelLDT = LinearDiscriminantAnalysis().fit(x_train_t,y_train_t)
predLinearT = modelLDT.predict(x_test_t)

with open("my_linearDiscAnalysis.sav", 'wb') as f:
    pickle.dump(modelLDT,f)
print(classification_report(y_test_t, predLinearT))
print(accuracy_score(y_test_t,predLinearT))

              precision    recall  f1-score   support

           0       0.81      0.85      0.83        41
           1       0.89      0.86      0.88        59

    accuracy                           0.86       100
   macro avg       0.85      0.86      0.86       100
weighted avg       0.86      0.86      0.86       100

0.86


In [15]:
#Set de entrenamiento Reddit
#x_train_r, x_test_r, y_train_r, y_test_r = train_test_split(X_reddit,Y_reddit, test_size = 0.2, random_state = 0)

In [16]:
#model = LinearDiscriminantAnalysis().fit(x_train_r,y_train_r)
#predLinearR = model.predict(x_test_r)
#print(classification_report(y_test_r, predLinearR))

In [17]:
#Set de entrenamiento Google
x_train_g, x_test_g, y_train_g, y_test_g = train_test_split(X_google,Y_google, test_size = 0.2, random_state = 0)

In [18]:
model = LinearDiscriminantAnalysis().fit(x_train_g,y_train_g)
predLinearG = model.predict(x_test_g)
print(classification_report(y_test_g, predLinearG))
print(accuracy_score(y_test_g,predLinearG))

              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.80      1.00      0.89         4

    accuracy                           0.83         6
   macro avg       0.90      0.75      0.78         6
weighted avg       0.87      0.83      0.81         6

0.8333333333333334


In [19]:
#Bagg of word para twitter
bow_vectorizerT = CountVectorizer(max_df=0.9, min_df=2, max_features=1000, stop_words='english')
bowT = bow_vectorizerT.fit_transform(dataSetCombinadoTwitter['Tweets'])


In [20]:
#Bagg of word para google
bow_vectorizerG = CountVectorizer(max_df=0.9, min_df=2, max_features=1000, stop_words='english')
bowG = bow_vectorizerG.fit_transform(dataSetCombinadoGoogle['Desc'])

In [21]:
#Bagg of word para reddit
#bow_vectorizerR = CountVectorizer(max_df=0.9, min_df=2, max_features=1000, stop_words='english')
#bowR = bow_vectorizerR.fit_transform(dataSetCombinadoReddit['Title'])

In [22]:
#Para entrenamiento de programa con modelo regresivo Twitter
x_train_t, x_test_t, y_train_t, y_test_t = train_test_split(bowT,dataSetCombinadoTwitter['Label'], random_state = 42, test_size=0.25)

In [23]:
#Para entrenamiento de programa con modelo regresivo Google
x_train_g, x_test_g, y_train_g, y_test_g = train_test_split(bowG,dataSetCombinadoGoogle['Label'], random_state = 42, test_size=0.25)

In [24]:
#Para entrenamiento de programa con modelo regresivo Reddit
#x_train, x_test, y_train, y_test = train_test_split(bowR,dataSetCombinado['Label'], random_state = 42, test_size=0.25)

In [25]:
modelLRT = LogisticRegression().fit(x_train_t,y_train_t)
predRegT = modelLRT.predict(x_test_t)
print(classification_report(y_test_t, predRegT))
print(accuracy_score(y_test_t,predRegT))

              precision    recall  f1-score   support

           0       0.65      0.66      0.65        53
           1       0.75      0.74      0.74        72

    accuracy                           0.70       125
   macro avg       0.70      0.70      0.70       125
weighted avg       0.70      0.70      0.70       125

0.704


In [26]:
modelLRG = LogisticRegression().fit(x_train_g,y_train_g)
predRegG = modelLRG.predict(x_test_g)
print(predRegG)
print(classification_report(y_test_g, predRegG))
print(accuracy_score(y_test_g,predRegG))

[1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.86      1.00      0.92         6

    accuracy                           0.86         7
   macro avg       0.43      0.50      0.46         7
weighted avg       0.73      0.86      0.79         7

0.8571428571428571


C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
#newPred = model.predict(df_tweet['Tweets'])

In [28]:
dataT = dataSetCombinadoTwitter['Tweets'].tolist()
lblT = dataSetCombinadoTwitter['Label'].tolist()

dataR = dataSetCombinadoReddit['Title'].tolist()
lblR = dataSetCombinadoReddit['Label'].tolist()

dataG = dataSetCombinadoGoogle['Desc'].tolist()
lblG = dataSetCombinadoGoogle['Label'].tolist()

In [29]:
#RandomForest
vectorizerT = TfidfVectorizer(max_features = 2500, max_df=0.8, stop_words = stopwords.words('english'))
dataT = vectorizerT.fit_transform(dataT).toarray()

#vectorizerR = TfidfVectorizer(max_features = 2500, max_df=0.8, stop_words = stopwords.words('english'))
#dataR = vectorizerR.fit_transform(dataR).toarray()

vectorizerG = TfidfVectorizer(max_features = 2500, max_df=0.8, stop_words = stopwords.words('english'))
dataG = vectorizerG.fit_transform(dataG).toarray()

In [30]:
x_train_t, x_test_t, y_train_t, y_test_t = train_test_split(dataT, lblT, test_size = 0.2, random_state = 0)

#x_train_r, x_test_r, y_train_r, y_test_r = train_test_split(dataR, lblR, test_size = 0.2, random_state = 0)

x_train_g, x_test_g, y_train_g, y_test_g = train_test_split(dataG, lblG, test_size = 0.2, random_state = 0)

In [31]:
text_classifierT = RandomForestClassifier(n_estimators = 200, random_state = 0)
text_classifierT.fit(x_train_t, y_train_t)

#text_classifierR = RandomForestClassifier(n_estimators = 200, random_state = 0)
#text_classifierR.fit(x_train_r, y_train_r)

text_classifierG = RandomForestClassifier(n_estimators = 200, random_state = 0)
text_classifierG.fit(x_train_g, y_train_g)

RandomForestClassifier(n_estimators=200, random_state=0)

In [32]:
predictionsT = text_classifierT.predict(x_test_t)

#predictionsR = text_classifierR.predict(x_test_r)

predictionsG = text_classifierG.predict(x_test_g)

In [33]:
print(confusion_matrix(y_test_t,predictionsT))
print(classification_report(y_test_t,predictionsT))
print(accuracy_score(y_test_t,predictionsT))

[[24 17]
 [37 22]]
              precision    recall  f1-score   support

           0       0.39      0.59      0.47        41
           1       0.56      0.37      0.45        59

    accuracy                           0.46       100
   macro avg       0.48      0.48      0.46       100
weighted avg       0.49      0.46      0.46       100

0.46


In [34]:
#print(confusion_matrix(y_test_r,predictionsR))
#print(classification_report(y_test_r,predictionsR))
#print(accuracy_score(y_test_r,predictionsR))

In [35]:
print(confusion_matrix(y_test_g,predictionsG))
print(classification_report(y_test_g,predictionsG))
print(accuracy_score(y_test_g,predictionsG))

[[0 2]
 [0 4]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.67      1.00      0.80         4

    accuracy                           0.67         6
   macro avg       0.33      0.50      0.40         6
weighted avg       0.44      0.67      0.53         6

0.6666666666666666


C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
#Gauseano
modelT = GaussianNB()
modelG = GaussianNB()
modelR = GaussianNB()

In [37]:
modelT.fit(x_train_t, y_train_t)
modelG.fit(x_train_g, y_train_g)

GaussianNB()

In [38]:
predictionT = modelT.predict(x_test_t)
predictionG = modelG.predict(x_test_g)

In [39]:
print(confusion_matrix(y_test_t,predictionsT))
print(classification_report(y_test_t,predictionsT))
print(accuracy_score(y_test_t,predictionsT))

[[24 17]
 [37 22]]
              precision    recall  f1-score   support

           0       0.39      0.59      0.47        41
           1       0.56      0.37      0.45        59

    accuracy                           0.46       100
   macro avg       0.48      0.48      0.46       100
weighted avg       0.49      0.46      0.46       100

0.46


In [40]:
print(confusion_matrix(y_test_g,predictionsG))
print(classification_report(y_test_g,predictionsG))
print(accuracy_score(y_test_g,predictionsG))

[[0 2]
 [0 4]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.67      1.00      0.80         4

    accuracy                           0.67         6
   macro avg       0.33      0.50      0.40         6
weighted avg       0.44      0.67      0.53         6

0.6666666666666666


C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
#Mejor Score
#Twitter -> Regression ~= 70
#Google -> Regression ~= 85

In [42]:
#Acceso a Twitter API
consumerKey = "TwlSxQ7X0qOJ9XPYi0RRGFmPy"
consumerSecret = "4QCBGcgWOOhHWXe0SwDu5UMVGVSZvnZujkHhacliUQeBigM7ZS"
accessToken = "1083093878893432836-HW9vv7t1xihZvZvq0YdomkNKPzFnWe"
accessTokenSecret = "PW8gxDGBCaNcprbkIb0uEQstg5tUzL55KRWEOI8Vqz2U9"

authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)
authenticate.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(authenticate, wait_on_rate_limit=True)


In [43]:
#LimpiaTexto
def cleanText(twt):
    EMOJI_PATTERN = re.compile(
    "["
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "\U000024C2-\U0001F251" 
    "]+"
    )
    twt = re.sub('#[A-Za-z0-9]+','',twt) #Removes any string with hash
    twt = re.sub(r'[0-9]+', '', twt)
    twt = re.sub('@[A-Za-z0-9]+','',twt) #Removes any string with at
    twt = re.sub('\\n','',twt) #Remove \n caracter
    twt = EMOJI_PATTERN.sub(r'', twt)
    twt = twt.replace('!','').replace('-','').replace('_','')
    twt = re.sub('https?:\/\/\S+','',twt) #Removes hastag
    return twt
def cleanTime(time):
    return time.strftime("%Y-%m-%d %H:%M")
def cleanTimeStamp(time):
    return datetime.fromtimestamp(time).strftime('%Y-%m-%d %H:%M')

In [44]:
analyzedCrypto = 'BTC'
#CREAR DATASET DE TWEETS
cantidadDeTweetsBuscados = 500
#loopeo para obtener y almacenar info de todas las monedas

search_term = analyzedCrypto+' -filter:retweets'
#mySince = datetime.now()-timedelta(minutes=30)
#mySince = mySince.strftime("%Y-%m-%d %H:%M:%S")
#myUntil = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
mySince="2021-07-2"
myUntil="2021-07-2"
tweets = tweepy.Cursor(api.search, q=search_term, lang ='en', since=mySince, until=myUntil, tweet_mode='extended').items(cantidadDeTweetsBuscados)
all_tweets = [tweet.full_text for tweet in tweets]

tweets = tweepy.Cursor(api.search, q=search_term, lang ='en', since=mySince, until=myUntil,  tweet_mode='extended').items(cantidadDeTweetsBuscados)
all_dates = [tweet.created_at for tweet in tweets]


formatoDataSetTweets = {'Date': all_dates, 'Tweets': all_tweets}
df_tweets = pd.DataFrame(data=formatoDataSetTweets)

df_tweets['Tweets'] = df_tweets['Tweets'].apply(cleanText)
df_tweets['Date'] = df_tweets['Date'].apply(cleanTime)

#Genero dataSet con valores financieros
api_v2 = btf.bitfinex_v2.api_v2()

#CREAR DATASET DE VALORES DE MERCADO
pair = analyzedCrypto
TIMEFRAME = '30m'

t_start = datetime(2021,6,2,0,0)
t_start = time.mktime(t_start.timetuple())*1000

t_stop = datetime(2021,7,2,0,0)
t_stop = time.mktime(t_stop.timetuple())*1000

result = api_v2.candles(symbol = 'BTCUSD', interval = TIMEFRAME, limit = 1000, start = t_start, end = t_stop)

camposDataFrame = ['Date','Open', 'Close', 'High', 'Low', 'Volume']
df_market = pd.DataFrame(result, columns=camposDataFrame)
df_market['Date'] = pd.to_datetime(df_market['Date'], unit='ms')
df_market['Date'] = df_market['Date'].apply(cleanTime)


googlenews=GoogleNews(start='07/2/2021',end='07/2/2021')
googlenews.search('Bitcoin')
result=googlenews.result()
camposDataFrame = ['title','media', 'datetime', 'desc']
df_news=pd.DataFrame(result,columns=camposDataFrame)
df_news['title']=df_news['title'].apply(cleanText)
df_news['desc']=df_news['desc'].apply(cleanText)
df_news.columns = ['Title','Media', 'Date', 'Desc']
df_news['Date'] = df_news['Date'].apply(cleanTime)

dataSetCombinadoGoogleActual = df_google.merge(df_market, how='inner',on='Date')
dataSetCombinadoTwitterActual = df_tweet.merge(df_market, how='inner', on='Date')

dataSetCombinadoTwitterActual = labeleador(dataSetCombinadoTwitter)
dataSetCombinadoGoogleActual = limpiaErroresG(dataSetCombinadoGoogle)
dataSetCombinadoGoogleActual = labeleador(dataSetCombinadoGoogle)



No keys, only access to public API functions


In [62]:
dataSetCombinadoTwitterActual.head(1)

,Date,Tweets,Subjectivity,Polarity,Label,Open,Close,High,Low,Volume,Compound,Negative,Neutral,Positive
0,2021-06-27 23:59,Best riskadjusted returns in the past hours:...,0.275,0.375,1,34660.0,34744.0,34750.0,34660.0,1.61210626,0.6369,0.0,0.682,0.318


In [52]:
with open("my_linearDiscAnalysis.sav",'rb') as f:
    newModel = pickle.load(f)

In [53]:
posT,negT,neuT,compT = [],[],[],[]
setSIA(dataSetCombinadoTwitterActual['Tweets'],posT,negT,neuT,compT)
dataSetCombinadoTwitterActual['Compound'] = compT
dataSetCombinadoTwitterActual['Negative'] = negT
dataSetCombinadoTwitterActual['Neutral'] = neuT
dataSetCombinadoTwitterActual['Positive'] = posT

In [96]:
keep_columns = ['Open','High','Low','Volume','Compound','Negative','Neutral','Positive']
trainDFLinealTwitter2 = dataSetCombinadoTwitterActual[keep_columns]
trainDFLinealTwitter2 = trainDFLinealTwitter2.head(1)

In [97]:
newModel.predict(trainDFLinealTwitter2)

array([1], dtype=int64)